In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-10-05 16:35:18--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.14’

input.txt.14        100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2023-10-05 16:35:18 (89.3 MB/s) - ‘input.txt.14’ saved [1115394/1115394]



In [2]:
import warnings
import pdb

In [3]:
warnings.filterwarnings("ignore")

In [4]:
!ls

input.txt     input.txt.11  input.txt.14  input.txt.4  input.txt.7  sample_data
input.txt.1   input.txt.12  input.txt.2   input.txt.5  input.txt.8
input.txt.10  input.txt.13  input.txt.3   input.txt.6  input.txt.9


In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np

In [6]:
text = open('./input.txt', 'rb').read().decode(encoding='utf-8')

In [7]:
print(len(text))
print(text[:250])

1115394
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [8]:
vocab = sorted(set(text))

In [9]:
print(len(vocab))
print(vocab)

65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [10]:
ids_from_chars = {i: id for id, i in enumerate(vocab)}
chars_from_ids = {id: i for id, i in enumerate(vocab)}

In [11]:
print(ids_from_chars)
print(chars_from_ids)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}
{0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i',

In [12]:
def text_from_ids(ids):
  text = ''
  for i in ids:
    text += chars_from_ids[i]

  return text

In [13]:
def text_to_tensor(start):
  chars = []
  for char in start:
    chars.append(ids_from_chars[char])

  return torch.tensor([chars])

In [14]:
text_to_tensor('hi')

tensor([[46, 47]])

In [15]:
text_from_ids([51, 53, 51, 1, 51, 53, 51])

'mom mom'

In [16]:
all_ids = [ids_from_chars[i] for i in text]

In [17]:
i = 0
sequences = []

while (i <= len(all_ids) // 100):
  sequences.append(all_ids[i:i + 101])
  i+= 101

sequences = torch.tensor(sequences)

In [18]:
sequences.shape

torch.Size([111, 101])

In [19]:
class MyDS(Dataset):
  def __init__(self, data):
    self.data = data
  def __len__(self):
    return len(self.data)
  def __getitem__(self, index):
    X = self.data[index][:-1]
    y = self.data[index][1:]
    return X, y

In [20]:
new_ds = MyDS(sequences)

In [21]:
dl = DataLoader(new_ds, batch_size=32, shuffle=True)

In [22]:
X, y = next(iter(dl))

In [23]:
text_from_ids(X.numpy()[0]),text_from_ids(y.numpy()[0])

('his country he did it to\nplease his mother and to be partly proud; which he\nis, even till the altitu',
 'is country he did it to\nplease his mother and to be partly proud; which he\nis, even till the altitud')

In [24]:
X.shape, y.shape

(torch.Size([32, 100]), torch.Size([32, 100]))

In [25]:
class MyModel(nn.Module):
  def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers):
    super(MyModel, self).__init__()

    self.hidden_size = hidden_size
    self.num_layers = num_layers

    self.embedding = nn.Embedding(vocab_size, embedding_dim)
    self.rnn = nn.RNN(input_size=embedding_dim,
                      hidden_size=hidden_size,
                      num_layers=num_layers,
                      batch_first=True)

    self.fc1 = nn.Linear(hidden_size, vocab_size)
  def forward(self, x):
    batch_size = x.size(0)

    hidden = self.init_hidden(batch_size)
    x = self.embedding(x)

    out, hidden = self.rnn(x, hidden)

    # pdb.set_trace()
    out = out.contiguous().view(-1, self.hidden_size)
    out = self.fc1(out)

    return out, hidden
  def init_hidden(self, batch_size):
    return torch.zeros(self.num_layers, batch_size, self.hidden_size)


In [26]:
def training_loop(model, dataloader, num_epochs, optimizer, loss_fn):
  losses = []
  for epoch in range(1, num_epochs + 1):
    for X, y in dataloader:

      y_hat, hidden = model(X)

      loss = loss_fn(y_hat, y.view(-1).long())

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if epoch % 10 == 0:
      print(epoch, num_epochs, float(loss))


In [27]:
vocab_size = len(vocab)

embedding_dim = 256

num_layers = 4

In [28]:
model = MyModel(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_size=256, num_layers=num_layers)

In [29]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)
loss_fn = nn.CrossEntropyLoss()

In [30]:
training_loop(model, dataloader=dl, num_epochs=100, optimizer=optimizer, loss_fn=loss_fn)

10 100 2.1780879497528076
20 100 1.7825469970703125
30 100 1.3815561532974243
40 100 1.0399137735366821
50 100 0.6719143986701965
60 100 0.5063482522964478
70 100 0.29081135988235474
80 100 0.1873340904712677
90 100 0.1123259887099266
100 100 0.10883636772632599


In [31]:
with torch.no_grad():
  model.eval()

  x = torch.tensor([[21, 22]])
  out, hidden = model(x)

  print(out.shape)
  print(chars_from_ids[int(torch.argmax(out[-1,:]).item())])

torch.Size([2, 65])
-


In [32]:
def predict(model, tensor):
  out, hidden = model(tensor)
  ids = int(torch.argmax(out[-1, :]).item())
  return ids

In [33]:
def sample(model, size, start):
  model.eval()

  sentence = text_to_tensor(start).tolist()

  with torch.no_grad():
    len_to_predict = size - len(sentence[0])

    for i in range(len_to_predict):
      pred_char = predict(model, torch.tensor(sentence))
      sentence[0].append(pred_char)

  return text_from_ids(sentence[0])

In [35]:
print(sample(model, 1000, 'MENENIUS:'))

MENENIUS:
For corn at their own rates; where go you
With bats and clubs:
Rome and her rats are at the point of battle;
The one side must have a lest were so!

MENENIUS:
What work's, my countrymen, in him. You must
Confess you lions, finds you hares;
Where foxes, geese: you are no surer, noble Marcius!

MARCIUS:
First unroof'd the city,
Ere so prevail'd with me: with these shreds
They'll sit by the fire, and presume to know
What's done i' the Capitol!

All:
Come, come.

First Citizen:
Ye're long about it.

MENENIUS:
Note me this, good friend;
Your most grave belly was now your hate,
Him vile that hath always loved
the people.

All:
We know't, we know't.

First Citizen:
Ye're long about it.

MENENIUS:
Note me this, good friend;
Your most grave belly was now your hate,
Him vile that hath always loved
the people.

All:
We know't, we know't.

First Citizen:
Ye're long about it.

MENENIUS:
Note me this, good friend;
Your most grave belly was now your hate,
Him vile that hath always loved
the